### Системы управления базами данных MongoDB и SQLite в Python

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pprint import pprint

In [2]:
def parser_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy,
        'area': '2',
        'clusters': 'true',
        'enable_snippets': 'true',
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }

    main_link = 'https://spb.hh.ru'

    html = requests.get(main_link + '/search/vacancy/', params=params, headers=headers)

    if html.ok:
        parsed_html = bs(html.text, 'lxml')

        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())

    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(main_link + '/search/vacancy/', params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text, 'lxml')
            vacancy_block = parsed_html.find('div', {'class': 'vacancy-serp'})
            vacancs_list = vacancy_block.findChildren(recursive = False)
                
            for item in vacancs_list:
                vacancy_date.append(parser_item_hh(item))
    
    return vacancy_date

In [3]:
def parser_item_hh(item):

    vacancy_date = {}
    
    tag_link = item.find('span', {'class': 'g-user-content'})
    if tag_link != None:
        tag_link_info = tag_link.findChild()
        vacancy_name = tag_link_info.getText()
        vacancy_link = tag_link_info['href']
        
        vacancy_date['vacancy_name'] = vacancy_name
        vacancy_date['vacancy_link'] = vacancy_link
              
    # company_name
    
        company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}).find('a').getText()
    
        vacancy_date['company_name'] = company_name
    
    # city
        city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText().split(', ')[0]
    
        vacancy_date['city'] = city
    
    #metro station
        metro_station = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()

        if not metro_station:
            metro_station = None
        else:
            metro_station = metro_station.getText()
        
        vacancy_date['metro_station'] = metro_station
    
    #salary
        salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if not salary:
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            salary = salary.getText().replace(u'\xa0', u'')
        
            salary = re.split(r'\s|-', salary)
        
            if salary[0] == 'до':
                salary_min = None
                salary_max = int(salary[1])
            elif salary[0] == 'от':
                salary_min = int(salary[1])
                salary_max = None
            else:
                salary_min = int(salary[0])
                salary_max = int(salary[1])            
        
            salary_currency = salary[2]
        
        vacancy_date['salary_min'] = salary_min
        vacancy_date['salary_max'] = salary_max
        vacancy_date['salary_currency'] = salary_currency
    
    # site
        vacancy_date['site'] = 'https://spb.hh.ru'
    
    return vacancy_date

In [4]:
def parser_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy,
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }
    
    main_link = 'https://russia.superjob.ru/vacancy/search/'
       
    html = requests.get(main_link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'lxml')
    
        page_block = parsed_html.find('div', {'class': "_3zucV L1p51 undefined _2guZ- _GJem"})
    if not page_block:
        last_page = 0
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page + 1
        html = requests.get(main_link, params=params, headers=headers)

        if html.ok:
            parsed_html = bs(html.text, 'lxml')
            vacancy_items = parsed_html.find_all('div', {'class': 'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})
                        
            for item in vacancy_items:
                vacancy_date.append(parser_item_superjob(item))
                
    return vacancy_date

In [5]:
def parser_item_superjob(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'}).text
    vacancy_date['vacancy_name'] = vacancy_name
    
    #vacancy_link
    vacancy_link = item.find('div', {'class': '_3mfro PlM3e _2JVkc _3LJqf'}).find('a')['href']
    vacancy_date['vacancy_link'] = 'https://russia.superjob.ru' + vacancy_link
    
    # company_name
    location = item.find('div', {'class': '_3_eyK _3P0J7 _9_FPy'})
    company_name = location.find('a')
    if not company_name:
        company_name = None
    else:
        company_name = company_name.text
    vacancy_date['company_name'] = company_name
    
    # city
    city = location.find('span', {'class': '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'}).text
    city = re.split(r'• |,', city)
    vacancy_date['city'] = city[1]
   
    #metro station
    if len(city) > 2:
        metro_station = city[2]
    else:
        metro_station = None
    vacancy_date['metro_station'] = metro_station
    
    #salary
    salary = item.find('span', {'class': '_3mfro _2Wp8I _1qw9T f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.text.replace(u'&nbsp;', u'')
        salary = re.split(r'\s|-', salary)
        if salary[0] == 'По':
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            if salary[0] == 'до':
                salary_min = None
                salary_max = int(salary[1] + salary[2])
                salary_currency = salary[3]
            elif salary[0] == 'от':
                salary_min = int(salary[1] + salary[2])
                salary_max = None
                salary_currency = salary[3]
            else:
                salary_min = int(salary[0] + salary[1])
                salary_max = int(salary[3] + salary[4])            
                salary_currency = salary[5]
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency

    # site
    vacancy_date['site'] = 'https://russia.superjob.ru'
    
    return vacancy_date

In [6]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(parser_hh(vacancy))
    vacancy_date.extend(parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)
    df_final = df.dropna(axis='index', how='any', subset=['vacancy_name'])

    return df_final

In [7]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [8]:
df

,vacancy_name,vacancy_link,company_name,city,metro_station,salary_min,salary_max,salary_currency,site
0,Data Scientist,https://spb.hh.ru/vacancy/37369353?query=Python,ЦРТ | Группа компаний,Санкт-Петербург,Выборгская,NaN,NaN,None,https://spb.hh.ru
1,Web-аналитик,https://spb.hh.ru/vacancy/37163724?query=Python,"Риалвеб, Интернет-агентство",Санкт-Петербург\n П...,\n Показать на карт...,NaN,NaN,None,https://spb.hh.ru
3,Marketing Analyst,https://spb.hh.ru/vacancy/37361879?query=Python,PRISMA,Санкт-Петербург,Василеостровская,NaN,NaN,None,https://spb.hh.ru
4,Разработчик DWH [id 21076],https://spb.hh.ru/vacancy/36308035?query=Python,ПАО «Газпром нефть»,Санкт-Петербург,Площадь Восстания,NaN,NaN,None,https://spb.hh.ru
5,Разработчик Python,https://spb.hh.ru/vacancy/37394225?query=Python,Vilesse,Санкт-Петербург\n П...,\n Показать на карт...,150000.0,200000.0,руб.,https://spb.hh.ru
...,...,...,...,...,...,...,...,...,...
1132,Главный разработчик,https://russia.superjob.ru/vakansii/glavnyj-ra...,NVI Solutions,Москва,Шаболовская,200000.0,NaN,руб.,https://russia.superjob.ru
1133,Сетевой инженер,https://russia.superjob.ru/vakansii/setevoj-in...,Retailpersonal,Ярославль,None,50000.0,60000.0,руб.,https://russia.superjob.ru
1134,"Web-программист, Web-разработчик (удалённо)",https://russia.superjob.ru/vakansii/web-progra...,Welbex,Москва,None,30000.0,NaN,руб.,https://russia.superjob.ru
1135,Программист C/С++,https://russia.superjob.ru/vakansii/programmis...,МФТИ ГУ,Москва,None,NaN,NaN,None,https://russia.superjob.ru


In [9]:
from pymongo import MongoClient

In [10]:
def save_to_db(data_frame, db_name):
    
    try:
        client = MongoClient('mongodb://127.0.0.1:27017')
        db = client[db_name]
        
        data = data_frame.to_dict('records')
        
        for item in data:
            db.vacancydb.insert_one(item)
        
        return f'Data saved to: {db_name}'
        
    except:
    
        return 'Something go wrong..'

In [96]:
save_to_db(df, vacancy)

'Data saved to: Python'

In [97]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client[vacancy]
vac = db.vacancydb
for v in vac.find({}):
    pprint(v)

{'_id': ObjectId('5ee0d0306d3e470e7bafe253'),
 'city': 'Санкт-Петербург',
 'company_name': ' ЦРТ | Группа компаний',
 'metro_station': 'Выборгская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37369353?query=Python',
 'vacancy_name': 'Data Scientist'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe254'),
 'city': 'Санкт-Петербург\n'
         '                            Показать на карте\n'
         '                        ',
 'company_name': ' Риалвеб, Интернет-агентство',
 'metro_station': '\n'
                  '                            Показать на карте\n'
                  '                        ',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37163724?query=Python',
 'vacancy_name': 'Web-аналитик'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe255'),
 'city': 'Санкт-Петербург',
 'company_name': ' PRISM

{'_id': ObjectId('5ee0d0306d3e470e7bafe2b8'),
 'city': 'Санкт-Петербург',
 'company_name': ' United Elements',
 'metro_station': 'Чкаловская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36773903?query=Python',
 'vacancy_name': 'Маркетолог-аналитик'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe2b9'),
 'city': 'Санкт-Петербург\n'
         '                            Показать на карте\n'
         '                        ',
 'company_name': ' Big Data Solutions',
 'metro_station': '\n'
                  '                            Показать на карте\n'
                  '                        ',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36967284?query=Python',
 'vacancy_name': 'DevOps Engineer (Cloud team)'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe2ba'),
 'city': 'Санкт-Петербург\n'
         '       

 'vacancy_link': 'https://spb.hh.ru/vacancy/36971316?query=Python',
 'vacancy_name': 'Chief Data Scientist'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe314'),
 'city': 'Санкт-Петербург',
 'company_name': 'Гос. корп. Военно-космическая академия имени А.Ф. Можайского',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/35870421?query=Python',
 'vacancy_name': 'Программист С++ / Linux'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe315'),
 'city': 'Санкт-Петербург',
 'company_name': ' АТОЛ, группа компаний',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36787482?query=Python',
 'vacancy_name': 'Инженер по тестированию ПО'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe316'),
 'city': 'Санкт-Петербург',
 'company_name': ' Ready for Sky',
 'metro_station': 'Нарвская',
 'salary_curr

 'vacancy_name': 'Ведущий тест-инженер (нагрузочное тестирование)'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe372'),
 'city': 'Санкт-Петербург',
 'company_name': ' Вяртсиля, Центр цифровых технологий',
 'metro_station': 'Василеостровская',
 'salary_currency': 'руб.',
 'salary_max': nan,
 'salary_min': 170000.0,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37286252?query=Python',
 'vacancy_name': 'Программист С++ (вычислительная геометрия)'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe373'),
 'city': 'Санкт-Петербург',
 'company_name': ' 585, Холдинг',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36724814?query=Python',
 'vacancy_name': 'Разработчик ASP.NET'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe374'),
 'city': 'Санкт-Петербург',
 'company_name': ' ARRIVAL LTD',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': n

{'_id': ObjectId('5ee0d0306d3e470e7bafe3e5'),
 'city': 'Санкт-Петербург',
 'company_name': ' Рексофт',
 'metro_station': 'Петроградская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36891350?query=Python',
 'vacancy_name': 'Software Engineer'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe3e6'),
 'city': 'Санкт-Петербург',
 'company_name': ' DINS',
 'metro_station': 'Елизаровская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36567119?query=Python',
 'vacancy_name': 'Senior DevOps engineer (DataOps) / Senior Data Operations '
                 'Engineer'}
{'_id': ObjectId('5ee0d0306d3e470e7bafe3e7'),
 'city': 'Санкт-Петербург',
 'company_name': 'ООО Стрим',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.h

...

{'_id': ObjectId('5ee1e07a6d3e470e7bafec87'),
 'city': 'Санкт-Петербург\n'
         '                            Показать на карте\n'
         '                        ',
 'company_name': 'ООО Бин64',
 'metro_station': '\n'
                  '                            Показать на карте\n'
                  '                        ',
 'salary_currency': 'руб.',
 'salary_max': 250000.0,
 'salary_min': 180000.0,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37008125?query=Python',
 'vacancy_name': 'С++ /Build Engineer'}
{'_id': ObjectId('5ee1e07a6d3e470e7bafec88'),
 'city': 'Санкт-Петербург',
 'company_name': ' JetBrains',
 'metro_station': 'Новокрестовская',
 'salary_currency': 'руб.',
 'salary_max': nan,
 'salary_min': 180000.0,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36863771?query=Python',
 'vacancy_name': 'DevOps Engineer (Business Applications Development)'}
{'_id': ObjectId('5ee1e07a6d3e470e7bafec89'),
 'city': 'Санкт

 'city': 'Санкт-Петербург',
 'company_name': ' DINS',
 'metro_station': 'Елизаровская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37238702?query=Python',
 'vacancy_name': 'Senior DevOps engineer (Contact Center)'}
{'_id': ObjectId('5ee1e07a6d3e470e7bafecf4'),
 'city': 'Санкт-Петербург',
 'company_name': ' Broadridge',
 'metro_station': 'Чкаловская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/34167154?query=Python',
 'vacancy_name': 'Senior Java Developer'}
{'_id': ObjectId('5ee1e07a6d3e470e7bafecf5'),
 'city': 'Санкт-Петербург',
 'company_name': ' Nexign (АО «Нэксайн»)',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37197639?query=Python',
 'vacancy_name': 'Junior DevOps special

 'city': 'Санкт-Петербург\n'
         '                            Показать на карте\n'
         '                        ',
 'company_name': ' Сбербанк для экспертов',
 'metro_station': '\n'
                  '                            Показать на карте\n'
                  '                        ',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37135913?query=Python',
 'vacancy_name': 'Разработчик Java+Python (команда NLP)'}
{'_id': ObjectId('5ee1e07b6d3e470e7bafed5e'),
 'city': 'Санкт-Петербург',
 'company_name': 'ПАО Ростелеком',
 'metro_station': 'Площадь Александра Невского 1',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37160606?query=Python',
 'vacancy_name': 'Ведущий инженер электросвязи'}
{'_id': ObjectId('5ee1e07b6d3e470e7bafed5f'),
 'city': 'Санкт-Петербург',
 'company_name': ' OctoBo

 'metro_station': 'Пионерская',
 'salary_currency': 'руб.',
 'salary_max': 110000.0,
 'salary_min': 70000.0,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37038928?query=Python',
 'vacancy_name': 'Инженер по тестированию / QA Engineer (сетевое оборудование)'}
{'_id': ObjectId('5ee1e07b6d3e470e7bafedc6'),
 'city': 'Санкт-Петербург',
 'company_name': 'ООО Специальный Технологический Центр',
 'metro_station': None,
 'salary_currency': 'руб.',
 'salary_max': 75000.0,
 'salary_min': 65000.0,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/35333632?query=Python',
 'vacancy_name': 'Инженер-программист (Python)'}
{'_id': ObjectId('5ee1e07b6d3e470e7bafedc7'),
 'city': 'Санкт-Петербург',
 'company_name': 'ООО Городской Психотерапевтический Центр Карповка-25',
 'metro_station': 'Петроградская',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/37037809?qu

{'_id': ObjectId('5ee1e07b6d3e470e7bafee35'),
 'city': 'Санкт-Петербург',
 'company_name': ' Лента, федеральная розничная сеть, IT ',
 'metro_station': 'Беговая',
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36935511?query=Python',
 'vacancy_name': 'BI аналитик'}
{'_id': ObjectId('5ee1e07b6d3e470e7bafee36'),
 'city': 'Санкт-Петербург',
 'company_name': 'ПАО МегаФон, Офис',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36709830?query=Python',
 'vacancy_name': 'Аналитик'}
{'_id': ObjectId('5ee1e07b6d3e470e7bafee37'),
 'city': 'Санкт-Петербург',
 'company_name': 'ООО Maxima',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': nan,
 'salary_min': nan,
 'site': 'https://spb.hh.ru',
 'vacancy_link': 'https://spb.hh.ru/vacancy/36923436?query=Python',
 'vacancy_name': 'М

2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

In [39]:
def search_in_db(db_name, salary):
    
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client[db_name]
    
    #objects = db.vacancydb.find( { $or: [ {'salary_min': {'$gt': salary}}, {'salary_max': {'$gt': salary}} ] } )
    #???не понятно почему на данныю команду выдает синтаксическую ошибку???
    
    objects = db.vacancydb.find( {'salary_min': {'$gt': salary}} )
                                        
    return list(objects)

In [41]:
search_in_db(vacancy, 120000)

[{'_id': ObjectId('5ee0d0306d3e470e7bafe257'),
  'vacancy_name': 'Разработчик Python',
  'vacancy_link': 'https://spb.hh.ru/vacancy/37394225?query=Python',
  'company_name': ' Vilesse',
  'city': 'Санкт-Петербург\n                            Показать на карте\n                        ',
  'metro_station': '\n                            Показать на карте\n                        ',
  'salary_min': 150000.0,
  'salary_max': 200000.0,
  'salary_currency': 'руб.',
  'site': 'https://spb.hh.ru'},
 {'_id': ObjectId('5ee0d0306d3e470e7bafe25d'),
  'vacancy_name': 'Python developer',
  'vacancy_link': 'https://spb.hh.ru/vacancy/36267290?query=Python',
  'company_name': ' B2Broker Санкт-Петербург',
  'city': 'Санкт-Петербург',
  'metro_station': 'Петроградская',
  'salary_min': 150000.0,
  'salary_max': 200000.0,
  'salary_currency': 'руб.',
  'site': 'https://spb.hh.ru'},
 {'_id': ObjectId('5ee0d0306d3e470e7bafe25e'),
  'vacancy_name': 'Python developer удаленно',
  'vacancy_link': 'https://spb

3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [126]:
def update_to_db(data_frame_new, db_name):
    
    try:
        client = MongoClient('mongodb://127.0.0.1:27017')
        db = client[db_name]
        vac = db.vacancydb
        
        data_new = data_frame_new.to_dict('records')
        
        #for item in data_new:
        #    vac.update_one({'vacancy_link': {'$in': [item['vacancy_link']]}}, {'$set': item}, {upsert: true})
        # ???в одну строку не работает, а код ниже работает в несколько строк тот же алгоритм???
        
        for item in data_new:
        #    search = vac.find_one({'vacancy_link': {'$in': [item['vacancy_link']]}})
            if bool(vac.find_one({'vacancy_link': {'$in': [item['vacancy_link']]}})):
                vac.update_one({'vacancy_link': item['vacancy_link']}, {'$set': item})
            else:
                vac.insert_one(item)
    
        return f'Data update to: {db_name}'
        
    except:
    
        return 'Something go wrong..'

In [127]:
vacancy = 'Python'
df_new = parser_vacancy(vacancy)
update_to_db(df_new, vacancy)

'Data update to: Python'